# Data Tranformation

## Import the packages

In [48]:
pip install python-amazon-paapi


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [49]:
import pandas as pd
import numpy as np
import requests
from concurrent.futures import ThreadPoolExecutor
import time

## Import the datas

In [62]:
repo_url = "https://raw.githubusercontent.com/baertsch/MGT-502-ML-Project/main/"
#items_df = pd.read_csv(repo_url + "/kaggle_data/items_df.csv")
items_df = pd.read_csv("./kaggle_data/items_df.csv")
display(items_df.head())

,Title,Author,ISBN Valid,Publisher,Subjects,i,Synopsis,Image
0,Classification décimale universelle : édition ...,['UDC Consortium (The Hague)'],9782871303336; 2871303339,Ed du CEFAL,Classification décimale universelle; Indexatio...,0,NaN,https://images.isbndb.com/covers/8447463483210...
1,Les interactions dans l'enseignement des langu...,"Cicurel, Francine, 1947-",9782278058327; 2278058320,Didier,didactique--langue étrangère - enseignement; d...,1,C'est dans l'interaction en classe que s'actua...,https://images.isbndb.com/covers/2099063482999...
2,Histoire de vie et recherche biographique : pe...,"['Aneta Slowik', 'Hervé Breton', 'Gaston Pineau']",2343190194; 9782343190198,L'Harmattan,Histoires de vie en sociologie; Sciences socia...,2,Depuis la parution en 1918 de l'ouvrage fondat...,NaN
3,Ce livre devrait me permettre de résoudre le c...,"Mazas, Sylvain, 1980-",9782365350020; 236535002X; 9782365350488; 2365...,Vraoum!,Moyen-Orient; Bandes dessinées autobiographiqu...,3,Product Description<br/><br/><br/>Sylvain Maza...,https://images.isbndb.com/covers/4356503483030...
4,Les années glorieuses : roman /,"Lemaitre, Pierre, 1951-",9782702180815; 2702180817; 9782702183618; 2702...,Calmann-Lévy,France--1945-1975; Roman historique; Roman fra...,4,"Trois histoires d'amour, un lanceur d'alerte, ...",https://images.isbndb.com/covers/2369653483150...


In [63]:
items_df.isna().sum()

Title            0
Author         402
ISBN Valid     723
Publisher       15
Subjects       475
i                0
Synopsis      5050
Image         4258
dtype: int64

In [52]:
#add a new column for the synopsis
#items_df['Synopsis'] = np.nan
items_df['Image'] = np.nan

## If time try to handle the duplicate

In [53]:
#items_df[items_df.duplicated(subset=['Title', 'Author'], keep=False)].sort_values('Title')

In [54]:
'''
# Group by 'Title' and 'Author' and track original indices
grouped = items_df.groupby(['Title', 'Author'], as_index=False).agg({
    'Publisher': lambda x: ', '.join(x.dropna().unique()),  # Combine unique publishers
    'ISBN Valid': lambda x: ', '.join(x.dropna().unique()),  # Combine unique ISBNs
    'Subjects': lambda x: ', '.join(x.dropna().unique()),  # Combine unique subjects
    # Collect original indices
    'index': lambda x: list(x.index)  # Track original indices
})

# Extract the mapping of original indices to new rows
index_mapping = grouped[['Title', 'Author', 'index']]

# Drop the 'index' column from the cleaned DataFrame
items_df_cleaned = grouped.drop(columns=['index'])

# Display the cleaned DataFrame and the index mapping
display(items_df_cleaned)
display(index_mapping)
'''

"\n# Group by 'Title' and 'Author' and track original indices\ngrouped = items_df.groupby(['Title', 'Author'], as_index=False).agg({\n    'Publisher': lambda x: ', '.join(x.dropna().unique()),  # Combine unique publishers\n    'ISBN Valid': lambda x: ', '.join(x.dropna().unique()),  # Combine unique ISBNs\n    'Subjects': lambda x: ', '.join(x.dropna().unique()),  # Combine unique subjects\n    # Collect original indices\n    'index': lambda x: list(x.index)  # Track original indices\n})\n\n# Extract the mapping of original indices to new rows\nindex_mapping = grouped[['Title', 'Author', 'index']]\n\n# Drop the 'index' column from the cleaned DataFrame\nitems_df_cleaned = grouped.drop(columns=['index'])\n\n# Display the cleaned DataFrame and the index mapping\ndisplay(items_df_cleaned)\ndisplay(index_mapping)\n"

## Filter the NaN values

In [55]:
data_na = items_df[((items_df['Author'].isna() )| (items_df['Subjects'].isna()) | (items_df['Publisher'].isna()) | (items_df['Synopsis'].isna()) | (items_df['Image'].isna())) & (items_df['ISBN Valid'].notna())]
display(data_na)

,Title,Author,ISBN Valid,Publisher,Subjects,i,Synopsis,Image
0,Classification décimale universelle : édition ...,['UDC Consortium (The Hague)'],9782871303336; 2871303339,Ed du CEFAL,Classification décimale universelle; Indexatio...,0,NaN,NaN
1,Les interactions dans l'enseignement des langu...,"Cicurel, Francine, 1947-",9782278058327; 2278058320,Didier,didactique--langue étrangère - enseignement; d...,1,C'est dans l'interaction en classe que s'actua...,NaN
2,Histoire de vie et recherche biographique : pe...,"['Aneta Slowik', 'Hervé Breton', 'Gaston Pineau']",2343190194; 9782343190198,L'Harmattan,Histoires de vie en sociologie; Sciences socia...,2,Depuis la parution en 1918 de l'ouvrage fondat...,NaN
3,Ce livre devrait me permettre de résoudre le c...,"Mazas, Sylvain, 1980-",9782365350020; 236535002X; 9782365350488; 2365...,Vraoum!,Moyen-Orient; Bandes dessinées autobiographiqu...,3,Product Description<br/><br/><br/>Sylvain Maza...,NaN
4,Les années glorieuses : roman /,"Lemaitre, Pierre, 1951-",9782702180815; 2702180817; 9782702183618; 2702...,Calmann-Lévy,France--1945-1975; Roman historique; Roman fra...,4,"Trois histoires d'amour, un lanceur d'alerte, ...",NaN
...,...,...,...,...,...,...,...,...
15285,Nouvelles orientales /,"Yourcenar, Marguerite",2070299732; 9782070299737,Gallimard,NaN,15285,NaN,NaN
15286,Le vagabond de Tokyo /,"Fukutani, Takashi, 1952-2000",9782353480111; 235348011X; 9782353480241; 2353...,Le Lézard noir,Mangas,15286,Née des cendres de la mythique revue undergrou...,NaN
15287,God of high school : le match contre les dieux /,"Park, Yong-Je",9782382880203; 2382880201; 9782382880210; 2382...,Kbooks,"Comics & Graphic Novels, Manga, Shonen",15287,NaN,NaN
15288,Blue Lock /,"Kaneshiro, Muneyuki",9782811650254; 2811650253; 9782811661274; 2811...,Pika,Compétitions; Football; Entraînement (sports);...,15288,"Coupe du monde 2018, l’équipe de football du J...",NaN


In [56]:
data_na.isna().sum()

Title             0
Author          237
ISBN Valid        0
Publisher         1
Subjects        255
i                 0
Synopsis       4327
Image         14568
dtype: int64

## API calls to fill nan
Uncomment the API you want to use

### ISBN DB

In [57]:
ISBNDB_API_KEY = '61388_3b6d517e1ce2bd51794dcfb3791558ad'

In [58]:
import requests

def fetch_metadata_isbndb_batch(isbn_list, api_key):
    headers = {
        'accept': 'application/json',
        'Authorization': api_key,
        'Content-Type': 'application/json',
    }
    
    data = 'isbns=' + ','.join(isbn_list)
    response = requests.post('https://api2.isbndb.com/books', headers=headers, data=(data))
    results = {}
    if response.status_code != 200:
        print(f"API error: {response.status_code} - {response.text}")
        return results
    try:
        data = response.json()
        for book in data.get("data", []):
            isbn_val = book.get("isbn13") or book.get("isbn")
            title = book.get("title", None)
            author = ", ".join(book.get("authors", [])) if book.get("authors") else np.nan
            publisher = book.get("publisher", None)
            synopsis = book.get("synopsis") or book.get("overview") or np.nan
            subjects = ", ".join(book.get("subjects", [])) if book.get("subjects") else np.nan
            image = book.get("image", np.nan)
            results[isbn_val] = (title, author, publisher, synopsis, subjects, image)
    except Exception as e:
        print(f"API error: {e}")
    return results

def update_metadata_isbndb_batch(data_na, api_key):
    unique_isbns = data_na['ISBN Valid'].dropna().apply(lambda x: x.split(';')[0].strip()).unique()
    metadata_mapping = {}
    # Batch in groups of 100
    for i in range(0, len(unique_isbns), 100):
        batch = unique_isbns[i:i+100]
        batch_metadata = fetch_metadata_isbndb_batch(batch, api_key)
        metadata_mapping.update(batch_metadata)
        time.sleep(1)  
    # Update DataFrame
    for index, row in data_na.iterrows():
        isbn = str(row.get('ISBN Valid')).split(';')[0].strip()
        if isbn in metadata_mapping:
            title, author, publisher, synopsis, subjects, image = metadata_mapping[isbn]
            if pd.isna(row['Title']):
                data_na.at[index, 'Title'] = title
            if pd.isna(row['Author']):
                data_na.at[index, 'Author'] = author
            if pd.isna(row['Publisher']):
                data_na.at[index, 'Publisher'] = publisher
            if pd.isna(row['Synopsis']):
                data_na.at[index, 'Synopsis'] = synopsis
            if pd.isna(row['Subjects']):
                data_na.at[index, 'Subjects'] = subjects
            if pd.isna(row['Image']):
                data_na.at[index, 'Image'] = image
    return data_na

# Usage:
data_na = update_metadata_isbndb_batch(data_na, ISBNDB_API_KEY)
display(data_na)

,Title,Author,ISBN Valid,Publisher,Subjects,i,Synopsis,Image
0,Classification décimale universelle : édition ...,['UDC Consortium (The Hague)'],9782871303336; 2871303339,Ed du CEFAL,Classification décimale universelle; Indexatio...,0,NaN,https://images.isbndb.com/covers/8447463483210...
1,Les interactions dans l'enseignement des langu...,"Cicurel, Francine, 1947-",9782278058327; 2278058320,Didier,didactique--langue étrangère - enseignement; d...,1,C'est dans l'interaction en classe que s'actua...,https://images.isbndb.com/covers/2099063482999...
2,Histoire de vie et recherche biographique : pe...,"['Aneta Slowik', 'Hervé Breton', 'Gaston Pineau']",2343190194; 9782343190198,L'Harmattan,Histoires de vie en sociologie; Sciences socia...,2,Depuis la parution en 1918 de l'ouvrage fondat...,NaN
3,Ce livre devrait me permettre de résoudre le c...,"Mazas, Sylvain, 1980-",9782365350020; 236535002X; 9782365350488; 2365...,Vraoum!,Moyen-Orient; Bandes dessinées autobiographiqu...,3,Product Description<br/><br/><br/>Sylvain Maza...,https://images.isbndb.com/covers/4356503483030...
4,Les années glorieuses : roman /,"Lemaitre, Pierre, 1951-",9782702180815; 2702180817; 9782702183618; 2702...,Calmann-Lévy,France--1945-1975; Roman historique; Roman fra...,4,"Trois histoires d'amour, un lanceur d'alerte, ...",https://images.isbndb.com/covers/2369653483150...
...,...,...,...,...,...,...,...,...
15285,Nouvelles orientales /,"Yourcenar, Marguerite",2070299732; 9782070299737,Gallimard,NaN,15285,NaN,NaN
15286,Le vagabond de Tokyo /,"Fukutani, Takashi, 1952-2000",9782353480111; 235348011X; 9782353480241; 2353...,Le Lézard noir,Mangas,15286,Née des cendres de la mythique revue undergrou...,https://images.isbndb.com/covers/2608636348302...
15287,God of high school : le match contre les dieux /,"Park, Yong-Je",9782382880203; 2382880201; 9782382880210; 2382...,Kbooks,"Comics & Graphic Novels, Manga, Shonen",15287,NaN,https://images.isbndb.com/covers/1114359348303...
15288,Blue Lock /,"Kaneshiro, Muneyuki",9782811650254; 2811650253; 9782811661274; 2811...,Pika,Compétitions; Football; Entraînement (sports);...,15288,"Coupe du monde 2018, l’équipe de football du J...",https://images.isbndb.com/covers/1718233483189...


### Google API Call with ISBN

In [ ]:
''' 
# Function to fetch metadata from Google Books API for a given ISBN
def fetch_metadata_google_books(isbn):
    try:
        # Extract the first part of the ISBN (before the semicolon)
        isbn = isbn.split(';')[0].strip()
        
        # Construct the API URL
        url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"
        response = requests.get(url).json()
        
        # Check if the API returned any results
        if "items" in response and len(response["items"]) > 0:
            book_data = response["items"][0]["volumeInfo"]
            author = book_data.get("authors", [np.nan])[0]
            categories = book_data.get("categories", [np.nan])[0]
            publisher = book_data.get("publisher", np.nan)
            description = book_data.get("description", np.nan)
        else:
            author, categories, publisher, description = np.nan, np.nan, np.nan, np.nan
    except Exception as e:
        print(f"Error fetching metadata for ISBN {isbn}: {e}")
        author, categories, publisher, description = np.nan, np.nan, np.nan, np.nan

    return author, categories, publisher, description

# Function to process a batch of ISBNs
def process_batch(isbn_list):
    results = []
    for isbn in isbn_list:
        results.append(fetch_metadata_google_books(isbn))
    return results

# Function to update the DataFrame with fetched metadata
def update_metadata(data_na):
    # Extract unique ISBNs to avoid redundant API calls
    unique_isbns = data_na['ISBN Valid'].dropna().unique()
    
    # Use ThreadPoolExecutor for parallel API calls
    with ThreadPoolExecutor(max_workers=5) as executor:
        results = list(executor.map(fetch_metadata_google_books, unique_isbns))
    
    # Create a mapping of ISBN to metadata
    metadata_mapping = {isbn: result for isbn, result in zip(unique_isbns, results)}
    
    # Update the DataFrame
    for index, row in data_na.iterrows():
        isbn = row.get('ISBN Valid')
        if pd.notna(isbn) and isbn in metadata_mapping:
            author, categories, publisher, description = metadata_mapping[isbn]
            if pd.isna(row['Author']):
                data_na.at[index, 'Author'] = author
            if pd.isna(row['Subjects']):
                data_na.at[index, 'Subjects'] = categories
            if pd.isna(row['Publisher']):
                data_na.at[index, 'Publisher'] = publisher
            if pd.isna(row['Synopsis']):
                data_na.at[index, 'Synopsis'] = description
    
    return data_na

# Example usage
data_na = update_metadata(data_na)
display(data_na)
'''

KeyboardInterrupt: 

### OpenLibrary API

In [ ]:
'''
# Function to fetch metadata from OpenLibrary API for a given ISBN
def fetch_metadata(isbn):
    try:
        # Extract the first part of the ISBN (before the semicolon)
        isbn = isbn.split(';')[0].strip()
        url = f"https://openlibrary.org/isbn/{isbn}.json"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            author = None
            if "authors" in data and data["authors"]:
                author_key = data["authors"][0]["key"]
                auth_response = requests.get(f"https://openlibrary.org{author_key}.json")
                if auth_response.status_code == 200:
                    author = auth_response.json().get("name")
            publisher = data.get("publishers", [None])[0]
            return author, publisher
    except Exception as e:
        print(f"Open Library error for ISBN {isbn}: {e}")
    return np.nan, np.nan, np.nan

    return author_name, publisher, subjects
# Function to process a batch of ISBNs
def process_batch(isbn_list):
    results = {}
    with ThreadPoolExecutor(max_workers=5) as executor:
        # Fetch metadata concurrently
        fetched_data = list(executor.map(fetch_metadata, isbn_list))
        for isbn, data in zip(isbn_list, fetched_data):
            results[isbn] = data
    return results

# Function to update the DataFrame with fetched metadata
def update_metadata(data_na):
    # Extract unique ISBNs to avoid redundant API calls
    unique_isbns = data_na['ISBN Valid'].dropna().unique()
    
    # Fetch metadata for all unique ISBNs
    metadata_mapping = process_batch(unique_isbns)
    
    # Update the DataFrame
    for index, row in data_na.iterrows():
        isbn = row.get('ISBN Valid')
        if pd.notna(isbn) and isbn in metadata_mapping:
            author, publisher, subjects = metadata_mapping[isbn]
            if pd.isna(row['Author']):
                data_na.at[index, 'Author'] = author
            if pd.isna(row['Publisher']):
                data_na.at[index, 'Publisher'] = publisher
    
    return data_na

# Example usage
data_na = update_metadata(data_na)
display(data_na)
'''

,Title,Author,ISBN Valid,Publisher,Subjects,i
26,"Pays basque : France, Espagne et Navarre",NaN,9782067253407; 2067253409,Michelin,Pays basque (Espagne); Navarre française (Fran...,26
47,La Fille à la moto /,"Suzuki, Ôji",9782889231164; 288923116X,Atrabile Editions,NaN,47
85,Monsieur Remarquable /,"Tokarczuk, Olga",8366789306; 9788366789302,Format,NaN,85
121,Le veilleur de lune /,"Bombace, Aurélie",9782017086864; 201708686X,Gautier-Languereau,NaN,121
122,"Schtinki, un sanglier au caractère de cochon :...","Giorgini, Victoria",9782970156505; 2970156504,Lunidea,NaN,122
...,...,...,...,...,...,...
15272,Oeuvres complètes /,"Michaux, Henri",2070114015; 9782070114016; 2070114023; 9782070...,Gallimard,NaN,15272
15282,L'homme sans qualités : roman /,"Musil, Robert",2020357607; 9782020357609; 2020357860; 9782020...,Ed du Seuil,NaN,15282
15285,Nouvelles orientales /,"Yourcenar, Marguerite",2070299732; 9782070299737,Gallimard,Modern fiction,15285
15287,God of high school : le match contre les dieux /,"Park, Yong-Je",9782382880203; 2382880201; 9782382880210; 2382...,Kbooks,NaN,15287


## Update the data

In [64]:
data_na.isna().sum()

Title            0
Author        1154
ISBN Valid     343
Publisher       22
Subjects      1691
i                0
dtype: int64

In [67]:
items_df.update(data_na)
items_df.isna().sum()

Title            0
Author        1154
ISBN Valid     723
Publisher       23
Subjects      1691
i                0
dtype: int64

In [16]:
# Save items_df to a CSV file
items_df.to_csv("./kaggle_data/items_df.csv", index=False)